In [ ]:
import os.path
import re
from pathlib import Path
import pandas as pd
from matplotlib import pyplot as plt
from pandas import DataFrame


In [ ]:
"""Load files into memory"""

# Baseline results
TEST_RESULT_FOLDER = Path("C:\\Users\\tizia\\PycharmProjects\\DDQN_Trading_MSC\\model_archive\\TestResults")
GRAPH_OUTPUT = Path("./graphs")
OPTIMIZATION_TESTS = TEST_RESULT_FOLDER.joinpath("Optimizing_DDQN_refined")
METRICS_RELATIVE = "final\\validation\\metrics.csv"

name_reg = re.compile(r"DDQN_os(\d+)_ws(\d+)")

required_columns = ['model_name', 'mode', 'op_space', 'window_size', 'return', 'total_volatility']

result_metrics = []
ref_bh = None
if os.path.exists(OPTIMIZATION_TESTS):
    test_directories = os.listdir(OPTIMIZATION_TESTS)
    print(f"Found {len(test_directories)} test folders:")
    print(test_directories)
    for i, test in enumerate(test_directories):
        test_path = OPTIMIZATION_TESTS.joinpath(test)
        for j, sample in enumerate(os.listdir(test_path)):
            sample_path = test_path.joinpath(sample).joinpath(METRICS_RELATIVE)
            sample_df = pd.read_csv(sample_path)
            if i == 0 and j == 0:
                ref_bh = sample_df.head(1)
            sample_df = sample_df.tail(2)
            sample_df['op_space'] = name_reg.search(test).group(1)
            sample_df['window_size'] = name_reg.search(test).group(2)
            sample_df['mode'] = sample_df['evaluation_mode'].apply(lambda x: "Eval" if x == True else "Train")
            sample_df['model_name'] = test.strip("DDQN_")
            sample_df = sample_df[required_columns]
            result_metrics.append(sample_df)

else:
    print(f"Folder: {OPTIMIZATION_TESTS} was not found. Nothing was loaded.")

metrics = pd.concat(result_metrics)
metrics = metrics.astype({'op_space': int, 'window_size': int})
metrics

In [ ]:
"""Make box plots and lines"""
metrics = metrics[metrics['mode'] == 'Train']


fig, ax = plt.subplots(figsize=(12, 5))
metrics.sort_values(by=['op_space', 'window_size'])
metrics.boxplot(column=['return'], by=['op_space', 'window_size'], ax=ax)
ax.axhline(y=ref_bh['return'][0], color='red', linestyle='dashed', label='Buy&Hold Return')
plt.legend()
plt.suptitle('')
plt.title('Return comparison')
plt.xlabel('Tuple of Observation space and Window size')
plt.ylabel('Return factor')
plt.savefig(GRAPH_OUTPUT.joinpath("op_ws_optimization.png"))
plt.close()

fig, ax = plt.subplots(figsize=(12, 5))
metrics.boxplot(column=['total_volatility'], by=['op_space', 'window_size'], ax=ax)
ax.axhline(y=ref_bh['total_volatility'][0], color='red', linestyle='dashed', label='Buy&Hold Volatility')
plt.legend()
plt.suptitle('')
plt.title('Volatility comparison')
plt.xlabel('Tuple of Observation space and Window size')
plt.ylabel('Total Volatility')
plt.savefig(GRAPH_OUTPUT.joinpath("vol_optimization.png"))
plt.close()
